In [ ]:
import pandas as pd
qt_df= pd.read_csv("qt_metrics.csv")
qt_df

In [ ]:
op_df= pd.read_csv("openstack_metrics.csv")
op_df

# 1. Merging dfs


In [ ]:
op_df['source'] = 'openstack'
qt_df['source'] = 'qt'
combined_df = pd.concat([op_df, qt_df], ignore_index=True)
combined_df

# handeling commits
Idea was to make a database, to determine average bug per commits, but as each commit is unique it seems that the commit:id is unqiue every time, then it means that we can nto use it to track one specifci persons commit patten.


In [ ]:

print(f"Total rows: {combined_df.shape[0]}")


print(f"Unique commit_id count: {combined_df['author_date'].nunique()}")


print(f"NaN in commit_id: {combined_df['author_date'].isna().sum()}")


In [ ]:
combined_df["revd"].value_counts()

# authour date and Handeling bug count

In [ ]:
combined_df['author_date'] = pd.to_datetime(combined_df['author_date'], unit='s')
combined_df

In [ ]:
combined_df['year'] = combined_df['author_date'].dt.year
combined_df['month'] = combined_df['author_date'].dt.month
combined_df['day_of_week'] = combined_df['author_date'].dt.dayofweek  # 0 = Monday, 6 = Sunday
combined_df['hour'] = combined_df['author_date'].dt.hour
def categorize_time(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return 'night'

combined_df['time_of_day'] = combined_df['hour'].apply(categorize_time)

combined_df = combined_df.sort_values(by='author_date')  # Sort by time
combined_df['time_since_last_commit'] = combined_df['author_date'].diff().dt.total_seconds()
combined_df['time_since_last_commit_hours'] = combined_df['time_since_last_commit'] / 3600  # Convert to hours
combined_df['time_since_last_commit_days'] = combined_df['time_since_last_commit'] / (3600 * 24)  # Convert to days
combined_df=combined_df.drop(columns=['time_since_last_commit','time_since_last_commit_hours'])





combined_df['time_since_last_commit_days'].fillna(0, inplace=True)
combined_df

In [ ]:
combined_df['bugcount'].value_counts()

As there is no indcation that missing bug counts are indicative of 0 bugs, I am going to remove them. I choose to do this over imputation as there are a lot of Nan in relation to other data.

In [ ]:
combined_df.dropna(subset=['bugcount'], inplace=True)
combined_df

In [ ]:
combined_df["bug"]= combined_df["bugcount"].apply(lambda x: 1 if x > 0 else 0)
columns = ["bug"] + [col for col in combined_df.columns if col != "bug"]
combined_df = combined_df[columns]
combined_df.drop(axis=0, columns= ['bugcount'])

In [ ]:
if 'day_of_week' in combined_df.columns:
    avg_bug_day = combined_df.groupby('day_of_week')['bug'].mean()
    print("Average Bug by Day of Week:")
    print(avg_bug_day)

if 'time_of_day' in combined_df.columns:
    avg_bug_time = combined_df.groupby('time_of_day')['bug'].mean()
    print("\nAverage Bug by Time of Day:")
    print(avg_bug_time)

if 'month' in combined_df.columns:
    avg_bug_time = combined_df.groupby('month')['bug'].mean()
    print("\nAverage Bug by month:")
    print(avg_bug_time)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
combined_df['time_of_day_encoded'] = label_encoder.fit_transform(combined_df['time_of_day'])

correlation_day = combined_df['day_of_week'].corr(combined_df['bugcount'])
correlation_time_label = combined_df['time_of_day_encoded'].corr(combined_df['bugcount'])
correlation_month = combined_df['month'].corr(combined_df['bugcount'])
correlation_year = combined_df['year'].corr(combined_df['bugcount'])

print(f"Correlation with day_of_week: {correlation_day}")
print(f"Correlation with time_of_day (label encoded): {correlation_time_label}")
print(f"Correlation with day_of_week: {correlation_month}")
print(f"Correlation with time_of_day (label encoded): {correlation_year}")


not worth keepign the time varaibles they have low correlation and low predictive power.

# NAs

In [ ]:
combined_df["fixcount"].value_counts()
#combined_df["fixcount"].isna().sum()

Reasonable assumption that if there are no fixes then nothing is inputed to fixcount. As such we will set NA values for fixcount = 0

In [ ]:
combined_df["fixcount"]= combined_df["fixcount"].fillna(0)

In [ ]:
na_counts = combined_df.isna().sum()
na_counts


check how many rows lost if we drop na

In [ ]:
cl_df=combined_df
cl_df.dropna(inplace=True)
cl_df

In [110]:
cl_df["revd"].value_counts()
cl_df=cl_df.drop(columns=["revd"])

In [ ]:

cl_df

In [ ]:
from sklearn.preprocessing import LabelEncoder

if 'time_of_day' in cl_df.columns:
    label_encoder = LabelEncoder()
    
    cl_df['time_of_day'] = label_encoder.fit_transform(cl_df['time_of_day'])
    
    label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print("Label encoding mapping:", label_mapping)
else:
    print("Column 'time_of_day' does not exist in the DataFrame.")



if 'source' in cl_df.columns:
    label_encoder = LabelEncoder()
    
    cl_df['source'] = label_encoder.fit_transform(cl_df['source'])
    
    label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print("Label encoding mapping:", label_mapping)
else:
    print("Column 'time_of_day' does not exist in the DataFrame.")
cl_df = cl_df.drop(columns=['time_of_day_encoded'])

# feature importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier
cl_df=cl_df.drop(columns= ["author_date", "commit_id",])

feature_columns = cl_df.drop(columns=['bug', 'bugcount','source']).columns
X = cl_df[feature_columns]
y = cl_df['bug']

rf = RandomForestClassifier(random_state=8)
rf.fit(X, y)

feature_importance = pd.Series(rf.feature_importances_, index=feature_columns).sort_values(ascending=False)
feature_importance




In [114]:
threshold = 0.1
important_features = feature_importance[feature_importance >= threshold].index
X = X[important_features]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = cl_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import xgboost
%pip install shap
import shap


X = cl_df.drop(columns=['bug', 'bugcount', 'source'])
y = cl_df['bug'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
model = xgboost.XGBClassifier().fit(X_train, y_train)
# model = RandomForestClassifier(random_state=8).fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

explainer = shap.Explainer(model)
shap_values = explainer(X_test)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=39)


In [ ]:
shap.plots.bar(shap_values, max_display=39)

# pca

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
           
pca = PCA()
pca.fit(X_train)

X_pca_transformed = pca.transform(X_train)

df_pca_components = pd.DataFrame(X_pca_transformed, columns=[f'PC{i+1}' for i in range(X_pca_transformed.shape[1])])





explained_variance = np.cumsum(pca.explained_variance_ratio_)

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--', color='b')
plt.title('Elbow Diagram: Explained Variance vs. Number of Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.show()



In [ ]:
pca = PCA()

pca.fit(X_train)

X_pca_transformed = pca.transform(X_train)

X_train_post_pca = pd.DataFrame(X_pca_transformed, columns=[f'PC{i+1}' for i in range(X_pca_transformed.shape[1])])

X_train_post_pca

In [ ]:
test_pca=pca.transform(X_test)
test_pca = pd.DataFrame(test_pca, columns=[f'PC{i+1}' for i in range(test_pca.shape[1])])
test_pca


In [ ]:
rf=RandomForestClassifier().fit(X_train_post_pca,y_train)


y_pred = rf.predict(test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

not doing pca, as it does not improve performance

# testing models with varying feature inclusion

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
!pip install catboost
from catboost import CatBoostClassifier
!pip install lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
!pip install ngboost

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
!pip install imbalanced-learn

!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

from imblearn.ensemble import RUSBoostClassifier

from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

!pip install rgf-python

from rgf.sklearn import RGFClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier

#from tensorflow.keras.models import Sequential
#rom tensorflow.keras.layers import Dense

!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
poly_log_reg = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2)),
    ('log_reg', LogisticRegression(max_iter=100000))
])
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
#!pip install deep-forest
#from deepforest import CascadeForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier

In [ ]:



models = [
    #('TabNetClassifier', TabNetClassifier(verbose=0)),
    ('XGBRFClassifier', XGBRFClassifier()),
    ('XGBClassifier', XGBClassifier()),
    ('Logistic Regression', LogisticRegression(max_iter=100000)),
    #('Cascade Forest', CascadeForestClassifier()),
    ('Gaussian Process Classifier', GaussianProcessClassifier()),
    ('Nearest Centroid', NearestCentroid()),
    ('Ridge Classifier', RidgeClassifier()),
    ('LightGBM DART', LGBMClassifier(boosting_type='dart')),
    ('LightGBM GOSS', LGBMClassifier(boosting_type='goss')),
    ('OneVsRest with SVC', OneVsRestClassifier(SVC())),
    ('OneVsOne with SVC', OneVsOneClassifier(SVC())),
    #('AutoSklearnClassifier', autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=600)),
    #('Multinomial Naive Bayes', MultinomialNB()),
    ('Polynomial Logistic Regression', poly_log_reg),
    ('Passive-Aggressive Classifier', PassiveAggressiveClassifier(max_iter=1000)),
    ('SGD Classifier', SGDClassifier(max_iter=100000)),
    # ('SVM Linear', SVC(kernel='linear')),
    # ('SVM Polynomial (degree 2)', SVC(kernel='poly', degree=2)),
    # ('SVM Polynomial (degree 3)', SVC(kernel='poly', degree=3)),
    # ('SVM Polynomial (degree 4)', SVC(kernel='poly', degree=4)),
    # ('SVM RBF', SVC(kernel='rbf')),
    # ('Linear SVC', LinearSVC(dual='auto', max_iter=100000)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Bernoulli Naive Bayes', BernoulliNB()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Extra Tree Classifier', ExtraTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Extra Trees Classifier', ExtraTreesClassifier()),
    ('Bagging Classifier', BaggingClassifier()),
    ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('CatBoostClassifier', CatBoostClassifier(verbose=0)),
    ('LGBMClassifier', LGBMClassifier()),
    ('HistGradientBoostingClassifier', HistGradientBoostingClassifier()),
    ('Perceptron', Perceptron(max_iter=100000)),
    ('BalancedRandomForestClassifier', BalancedRandomForestClassifier()),
    ('BalancedBaggingClassifier', BalancedBaggingClassifier()),
    ('CalibratedClassifierCV', CalibratedClassifierCV(estimator=LogisticRegression())),
    ('MLPClassifier', MLPClassifier(max_iter=100000)),
    ('RUSBoostClassifier', RUSBoostClassifier()),
    ('RGFClassifier', RGFClassifier()),
    ('Voting Classifier', VotingClassifier(estimators=[
        ('lr', LogisticRegression()), 
        ('rf', RandomForestClassifier())
        # ('svc', SVC(kernel='linear'))
    ])),
    ('Stacking Classifier', StackingClassifier(estimators=[
        ('rf', RandomForestClassifier()),
        ('knn', KNeighborsClassifier())
    ], final_estimator=LogisticRegression()))
]

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=8)

results = []

for name, model in models:
    if name == 'TabNetClassifier':
        X_train_np = X_train.values  # Convert DataFrame to NumPy array
        cv_scores = cross_val_score(model, X_train_np, y_train, cv=kf, scoring='accuracy')
    else:
        cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')

    mean_accuracy = np.mean(cv_scores)
    print(f"{name} accuracy: {mean_accuracy} ")
    std_accuracy = np.std(cv_scores)

    results.append((name, mean_accuracy, std_accuracy))

sorted_results = sorted(results, key=lambda item: item[1], reverse=True)

summary = []
for rank, (model_name, mean_accuracy, std_accuracy) in enumerate(sorted_results):
    summary.append([model_name, rank + 1, mean_accuracy, std_accuracy])

summary_df = pd.DataFrame(summary, columns=['Model', 'Rank', 'Mean Accuracy', 'Std Deviation'])

summary_df



In [27]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

# normalised

In [ ]:
X

In [ ]:
from sklearn.preprocessing import MinMaxScaler

all_columns = X.columns

categorical_columns = ['time_of_day', 'day_of_week', 'month', 'hour', 'year', 'time_of_day_encoded']
numerical_columns = [col for col in all_columns if col not in categorical_columns]

scaler = MinMaxScaler()

X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

X.head()


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)


In [ ]:


models = [
    #('TabNetClassifier', TabNetClassifier(verbose=0)),
    ('XGBRFClassifier', XGBRFClassifier()),
    ('XGBClassifier', XGBClassifier()),
    ('Logistic Regression', LogisticRegression(max_iter=100000)),
    #('Cascade Forest', CascadeForestClassifier()),
    ('Gaussian Process Classifier', GaussianProcessClassifier()),
    ('Nearest Centroid', NearestCentroid()),
    ('Ridge Classifier', RidgeClassifier()),
    ('LightGBM DART', LGBMClassifier(boosting_type='dart')),
    ('LightGBM GOSS', LGBMClassifier(boosting_type='goss')),
    ('OneVsRest with SVC', OneVsRestClassifier(SVC())),
    ('OneVsOne with SVC', OneVsOneClassifier(SVC())),
    #('AutoSklearnClassifier', autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=600)),
    #('Multinomial Naive Bayes', MultinomialNB()),
    ('Polynomial Logistic Regression', poly_log_reg),
    ('Passive-Aggressive Classifier', PassiveAggressiveClassifier(max_iter=1000)),
    ('SGD Classifier', SGDClassifier(max_iter=100000)),
    ('SVM Linear', SVC(kernel='linear')),
    ('SVM Polynomial (degree 2)', SVC(kernel='poly', degree=2)),
    ('SVM Polynomial (degree 3)', SVC(kernel='poly', degree=3)),
    ('SVM Polynomial (degree 4)', SVC(kernel='poly', degree=4)),
    ('SVM RBF', SVC(kernel='rbf')),
    ('Linear SVC', LinearSVC(dual='auto', max_iter=100000)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Bernoulli Naive Bayes', BernoulliNB()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Extra Tree Classifier', ExtraTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Extra Trees Classifier', ExtraTreesClassifier()),
    ('Bagging Classifier', BaggingClassifier()),
    ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('CatBoostClassifier', CatBoostClassifier(verbose=0)),
    ('LGBMClassifier', LGBMClassifier()),
    ('HistGradientBoostingClassifier', HistGradientBoostingClassifier()),
    ('Perceptron', Perceptron(max_iter=100000)),
    ('BalancedRandomForestClassifier', BalancedRandomForestClassifier()),
    ('BalancedBaggingClassifier', BalancedBaggingClassifier()),
    ('CalibratedClassifierCV', CalibratedClassifierCV(estimator=LogisticRegression())),
    # ('MLPClassifier', MLPClassifier(max_iter=100000)),
    ('RUSBoostClassifier', RUSBoostClassifier()),
    ('RGFClassifier', RGFClassifier()),
    ('Voting Classifier', VotingClassifier(estimators=[
        ('lr', LogisticRegression()), 
        ('rf', RandomForestClassifier())
        # ('svc', SVC(kernel='linear'))
    ])),
    ('Stacking Classifier', StackingClassifier(estimators=[
        ('rf', RandomForestClassifier()),
        ('knn', KNeighborsClassifier())
    ], final_estimator=LogisticRegression()))
]

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=8)

results = []

for name, model in models:
    if name == 'TabNetClassifier':
        X_train_np = X_train.values  # Convert DataFrame to NumPy array
        cv_scores = cross_val_score(model, X_train_np, y_train, cv=kf, scoring='accuracy')
    else:
        cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')

    mean_accuracy = np.mean(cv_scores)
    print(f"{name} accuracy: {mean_accuracy} ")
    std_accuracy = np.std(cv_scores)

    results.append((name, mean_accuracy, std_accuracy))

sorted_results = sorted(results, key=lambda item: item[1], reverse=True)

summary = []
for rank, (model_name, mean_accuracy, std_accuracy) in enumerate(sorted_results):
    summary.append([model_name, rank + 1, mean_accuracy, std_accuracy])

summary_df = pd.DataFrame(summary, columns=['Model', 'Rank', 'Mean Accuracy', 'Std Deviation'])

summary_df




In [48]:
X_train.to_csv('X_train_norm.csv', index=False)
X_test.to_csv('X_test_norm.csv', index=False)


# pca + normalised

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

combined_data = pd.concat([X_train_post_pca, test_pca], axis=0)

scaler = StandardScaler()
normalized_data = scaler.fit_transform(combined_data)

X_train_normalized = normalized_data[:len(X_train_post_pca)] 
X_test_normalized = normalized_data[len(X_train_post_pca):]  

X_train_normalized_df = pd.DataFrame(X_train_normalized, columns=X_train_post_pca.columns)
X_test_normalized_df = pd.DataFrame(X_test_normalized, columns=test_pca.columns)

print("X_train_normalized shape:", X_train_normalized_df.shape)
print("X_test_normalized shape:", X_test_normalized_df.shape)


In [ ]:
seed = 8
models = [
    #('TabNetClassifier', TabNetClassifier(verbose=0)),
    ('XGBRFClassifier', XGBRFClassifier(random_seed=seed)),
    ('XGBClassifier', XGBClassifier(random_seed=seed)),
    ('Logistic Regression', LogisticRegression(max_iter=100000)),
    #('Cascade Forest', CascadeForestClassifier(random_seed=seed)),
    ('Gaussian Process Classifier', GaussianProcessClassifier()),
    ('Nearest Centroid', NearestCentroid()),
    ('Ridge Classifier', RidgeClassifier()),
    ('LightGBM DART', LGBMClassifier(boosting_type='dart', random_seed=seed)),
    ('LightGBM GOSS', LGBMClassifier(boosting_type='goss', random_seed=seed)),
    ('OneVsRest with SVC', OneVsRestClassifier(SVC())),
    ('OneVsOne with SVC', OneVsOneClassifier(SVC())),
    #('AutoSklearnClassifier', autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=600)),
    #('Multinomial Naive Bayes', MultinomialNB()),
    ('Polynomial Logistic Regression', poly_log_reg),
    ('Passive-Aggressive Classifier', PassiveAggressiveClassifier(max_iter=1000)),
    ('SGD Classifier', SGDClassifier(max_iter=100000)),
    ('SVM Linear', SVC(kernel='linear', )),
    ('SVM Polynomial (degree 2)', SVC(kernel='poly', degree=2, random_state=seed)),
    ('SVM Polynomial (degree 3)', SVC(kernel='poly', degree=3, random_state=seed)),
    ('SVM Polynomial (degree 4)', SVC(kernel='poly', degree=4, random_state=seed)),
    ('SVM RBF', SVC(kernel='rbf', random_state=seed)),
    ('Linear SVC', LinearSVC(dual='auto', max_iter=100000, random_state=seed)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Bernoulli Naive Bayes', BernoulliNB()),
    ('Decision Tree', DecisionTreeClassifier(random_state=seed)),
    ('Extra Tree Classifier', ExtraTreeClassifier(random_state=seed)),
    ('Random Forest', RandomForestClassifier(random_state=seed)),
    ('AdaBoost', AdaBoostClassifier(random_state=seed)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=seed)),
    ('Extra Trees Classifier', ExtraTreesClassifier(random_state=seed)),
    ('Bagging Classifier', BaggingClassifier(random_state=seed)),
    ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('CatBoostClassifier', CatBoostClassifier(random_seed=seed, verbose=0)),
    ('LGBMClassifier', LGBMClassifier(random_seed=seed)),
    ('HistGradientBoostingClassifier', HistGradientBoostingClassifier(random_state=seed)),
    ('Perceptron', Perceptron(max_iter=100000)),
    ('BalancedRandomForestClassifier', BalancedRandomForestClassifier(random_state=seed)),
    ('BalancedBaggingClassifier', BalancedBaggingClassifier(random_state=seed)),
    ('CalibratedClassifierCV', CalibratedClassifierCV(estimator=LogisticRegression(random_state=seed))),
    # ('MLPClassifier', MLPClassifier(max_iter=100000)),
    ('RUSBoostClassifier', RUSBoostClassifier(random_state=seed)),
    ('RGFClassifier', RGFClassifier()),
    ('Voting Classifier', VotingClassifier(estimators=[
        ('lr', LogisticRegression(random_state=seed)), 
        ('rf', RandomForestClassifier(random_state=seed))
        # ('svc', SVC(kernel='linear'))
    ])),
    ('Stacking Classifier', StackingClassifier(estimators=[
        ('rf', RandomForestClassifier(random_state=seed)),
        ('knn', KNeighborsClassifier())
    ], final_estimator=LogisticRegression()))
]
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=8)

results = []

for name, model in models:
    if name == 'TabNetClassifier':
        X_train_np = X_train.values  # Convert DataFrame to NumPy array
        cv_scores = cross_val_score(model, X_train_normalized_df, y_train, cv=kf, scoring='accuracy')
    else:
        cv_scores = cross_val_score(model, X_train_normalized_df, y_train, cv=kf, scoring='accuracy')

    mean_accuracy = np.mean(cv_scores)
    print(f"{name} accuracy: {mean_accuracy} ")
    std_accuracy = np.std(cv_scores)

    results.append((name, mean_accuracy, std_accuracy))

sorted_results = sorted(results, key=lambda item: item[1], reverse=True)

summary = []
for rank, (model_name, mean_accuracy, std_accuracy) in enumerate(sorted_results):
    summary.append([model_name, rank + 1, mean_accuracy, std_accuracy])

summary_df = pd.DataFrame(summary, columns=['Model', 'Rank', 'Mean Accuracy', 'Std Deviation'])

summary_df

In [ ]:
X_train_normalized_df.to_csv('X_train_norm_pca.csv', index=False)
X_test_normalized_df.to_csv('X_test_norm_pca.csv', index=False)
X_train_normalized_df